In [83]:
import pandas as pd
from sklearn.model_selection import train_test_split
from LinearRegression import run_linear_regression
from RandomForestRegressor import RandomForestRegressor
from XGBoostRegressor import XGBoostRegressor
import numpy as np
from sklearn.metrics import root_mean_squared_error, r2_score

In [84]:
df_news = pd.read_csv('/Users/oryining/Documents/NUS MComp/Semester 2/CS5344 - Big Data Analytics Technology/CS5344_Project/1. Data Retrieval and Processing of News and Stock Prices/final_sample_finbert.csv')
columns_to_select = ['Date_hourly', 'weighted_positive_fb', 'weighted_negative_fb', 'weighted_neutral_fb','weighted_DocTone']
df_news = df_news[columns_to_select]
df_news['Date_hourly'] = df_news['Date_hourly'].astype(str).str.strip()

In [85]:
df_stock_price = pd.read_csv('/Users/oryining/Documents/NUS MComp/Semester 2/CS5344 - Big Data Analytics Technology/CS5344_Project/1. Data Retrieval and Processing of News and Stock Prices/us_2024_ticker combined versions/SPY_Combined_WithPriceChange.csv',index_col=0)

df_stock_price['<DATE>'] = df_stock_price['<DATE>'].astype(str)
df_stock_price['<TIME>'] = df_stock_price['<TIME>'].astype(str).str.zfill(6)

df_stock_price['Date_hourly'] = '20' + df_stock_price['<DATE>'] + df_stock_price['<TIME>'].str[:2]
df_stock_price=df_stock_price.iloc[:, -2:]

In [86]:
# Merge the datasets on the 'Date_hourly' column using an inner join to keep only matching rows
merged_df = pd.merge(df_news, df_stock_price, on='Date_hourly', how='inner')
merged_df

,Date_hourly,weighted_positive_fb,weighted_negative_fb,weighted_neutral_fb,weighted_DocTone,PriceChange
0,2024010210,0.315752,0.280714,0.604901,0.461378,-0.134456
1,2024010211,0.425572,0.419274,0.395721,0.287102,0.318019
2,2024010212,0.309108,0.270425,0.627370,0.427144,-0.116232
3,2024010213,0.245346,0.273890,0.768499,0.733780,-0.116365
4,2024010214,0.334500,0.321078,0.455962,0.392139,-0.248875
...,...,...,...,...,...,...
1020,2024101710,0.447046,0.426239,0.552215,0.399509,-0.188298
1021,2024101711,0.202347,0.396428,0.607876,0.420399,0.265789
1022,2024101712,0.384186,0.255446,0.500886,0.456970,-0.059002
1023,2024101713,0.244025,0.520990,0.531706,0.254986,-0.003422


In [87]:
# Train Test Split and creating of x_train, x_val, x and y_train, y_val, y_test
Y = merged_df[['PriceChange']]  
X = merged_df[['weighted_positive_fb', 'weighted_negative_fb', 'weighted_neutral_fb','weighted_DocTone']]  
x_train_val, x_test, y_train_val, y_test = train_test_split(X, Y, test_size=0.2, random_state=1616)
x_train, x_val, y_train, y_val = train_test_split(x_train_val, y_train_val, test_size=0.25, random_state=1616)  

### Individual Models to input to Ensemble

In [88]:
def LinearRegressionScratch():
    #Linear Regresion Model
    lin_reg_model = run_linear_regression(x_train, y_train)

    # Make predictions
    lin_reg_predicted_values = lin_reg_model.predict(x_test.values)

    train_score = lin_reg_model.score(x_train.values, y_train.values.ravel())
    test_score = lin_reg_model.score(x_test.values, y_test.values.ravel())
    train_rmse = lin_reg_model.rmse(x_train.values, y_train.values.ravel())
    test_rmse = lin_reg_model.rmse(x_test.values, y_test.values.ravel())

    print(f"Training RMSE: {train_rmse}")
    print(f"Test RMSE: {test_rmse}")


    return lin_reg_predicted_values,lin_reg_model


In [89]:
#Random Forest Regressor 
def RandomForestRegressorScratch():

    rfr_model = RandomForestRegressor(n_estimators=250, max_depth=5, min_samples_split=8)
    rfr_model.fit(x_train.values, y_train.values.ravel())
    rfr_predicted_values = rfr_model.predict(x_test.values)

    train_score = rfr_model.score(x_train.values, y_train.values.ravel())
    test_score = rfr_model.score(x_test.values, y_test.values.ravel())
    train_rmse = rfr_model.rmse(x_train.values, y_train.values.ravel())
    test_rmse = rfr_model.rmse(x_test.values, y_test.values.ravel())

    print(f"Training RMSE: {train_rmse}")
    print(f"Test RMSE: {test_rmse}")

    return rfr_predicted_values,rfr_model

In [90]:
#XGBoost Regressor  
def XGBoostRegressorScratch():

    xgb_model = XGBoostRegressor(n_estimators=100, learning_rate=0.2, max_depth=8)
    xgb_model.fit(x_train.values, y_train.values.ravel())
    # Make predictions on the test set
    xgb_predicted_values = xgb_model.predict(x_test.values)

    train_score = xgb_model.score(x_train.values, y_train.values.ravel())
    test_score = xgb_model.score(x_test.values, y_test.values.ravel())
    train_rmse = xgb_model.rmse(x_train.values, y_train.values.ravel())
    test_rmse = xgb_model.rmse(x_test.values, y_test.values.ravel())

    print(f"Training RMSE: {train_rmse}")
    print(f"Test RMSE: {test_rmse}")

    return xgb_predicted_values,xgb_model

### Ensemble Model

In [93]:
print("===============Self-Built Linear Regresion Outputs=====================")
lin_reg_predicted_values, lin_reg_model= LinearRegressionScratch()
print("===============Self-Built RandomForest Regressor Outputs================")
rfr_predicted_values,rfr_model = RandomForestRegressorScratch()
print("===============Self-Built XGBoost Regressor Baseline Outputs============")
xgb_predicted_values,xgb_model = XGBoostRegressorScratch()

#Model predictions from X_test that are combined to be passed into the ensemble
y_pred_stacked_predictions = np.column_stack((lin_reg_predicted_values, rfr_predicted_values,xgb_predicted_values))
y_pred_stacked_predictions_df = pd.DataFrame(y_pred_stacked_predictions, columns=['reg_pred1', 'rf_pred2','xgboost_pred3'])

#Generating Y_validation values from X_values
y_val_input_reg_model = lin_reg_model.predict(x_val.values)
y_val_input_rf_model=rfr_model.predict(x_val.values)
y_val_input_xgb_model=xgb_model.predict(x_val.values)

y_val_input_model_predictions = np.column_stack((y_val_input_reg_model, y_val_input_rf_model,y_val_input_xgb_model))
y_val_input_model_predictions_df = pd.DataFrame(y_val_input_model_predictions, columns=['reg_pred1', 'rf_pred2','xgboost_pred3'])

ensemble_model= run_linear_regression(y_pred_stacked_predictions_df,y_test) # Predicted Values of the Models, and y_test is the Y variable
y_ensemble = ensemble_model.predict(y_val_input_model_predictions_df) 

val_rmse = root_mean_squared_error(y_ensemble, y_val)
r2 = r2_score(y_ensemble, y_val)

print("===============Self-Built Ensemble Model============================")
print(f'Validation RMSE: {val_rmse}')
print(ensemble_model.weights)



===============Self-Built Linear Regresion Outputs=====================
Training RMSE: 0.2344863421847283
Test RMSE: 0.22507911350244547
===============Self-Built RandomForest Regressor Outputs================
Training RMSE: 0.20885851337753172
Test RMSE: 0.2259210308835809
===============Self-Built XGBoost Regressor Baseline Outputs============
Training RMSE: 0.005177653338509913
Test RMSE: 0.2447207384730021
===============Self-Built Ensemble Model============================
Validation RMSE: 0.22098986597268735
[1.78876689 0.20240072 0.112561  ]
